In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tensorflow import keras
import tensorflow as tf
from tensorflow import keras 

In [3]:
IMAGE_SIZE = 256
BATCH_SIZE = 32
CHANNELS = 3

# Pepper,Tomato,Potato 

In [4]:
def CNN(n_classes):
    input_shape = (BATCH_SIZE, IMAGE_SIZE, IMAGE_SIZE, CHANNELS)

    model = tf.keras.Sequential([
        resize_and_rescale,
        keras.layers.Conv2D(32, kernel_size = (3,3), activation='relu', input_shape=input_shape),
        keras.layers.MaxPooling2D((2, 2)),
        keras.layers.Conv2D(64,  kernel_size = (3,3), activation='relu'),
        keras.layers.MaxPooling2D((2, 2)),
        keras.layers.Conv2D(64,  kernel_size = (3,3), activation='relu'),
        keras.layers.MaxPooling2D((2, 2)),
        keras.layers.Conv2D(64, (3, 3), activation='relu'),
        keras.layers.MaxPooling2D((2, 2)),
        keras.layers.Conv2D(64, (3, 3), activation='relu'),
        keras.layers.MaxPooling2D((2, 2)),
        keras.layers.Conv2D(64, (3, 3), activation='relu'),
        keras.layers.MaxPooling2D((2, 2)),
        keras.layers.Flatten(),
        keras.layers.Dense(64, activation='relu'),
        keras.layers.Dense(n_classes, activation='softmax'),
    ])

    model.build(input_shape=input_shape)


    model.compile(
        optimizer="adam",
        loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
        metrics="accuracy"
    )
    
    return model

def TrainCNN(model,trainset,valset,epochs):
    model.fit(trainset,epochs=epochs,batch_size=BATCH_SIZE,verbose=1,validation_data=valset)    

In [5]:
dataset = tf.keras.preprocessing.image_dataset_from_directory(
    "PlantVillage",
    shuffle = True,
    image_size = (IMAGE_SIZE,IMAGE_SIZE),
    batch_size = BATCH_SIZE,
)
CLASS_NAMES = dataset.class_names
print("Class names : ",CLASS_NAMES)
print(dataset.class_names)

Found 48400 files belonging to 3 classes.
Class names :  ['Pepper', 'Potato', 'Tomato']
['Pepper', 'Potato', 'Tomato']


In [6]:
print("number of  batchs",len(dataset))
leng = len(dataset)
tr_size = int(leng*0.8)

trainset = dataset.take(tr_size)
rest = dataset.skip(tr_size)
testset = rest.take(int((leng-tr_size)/2))
valset = rest.skip(int((leng-tr_size)/2))

print("TrainSet : number of  batchs",len(trainset))
print("TestSet : number of  batchs",len(testset))
print("ValSet : number of  batchs",len(valset))

number of  batchs 1513
TrainSet : number of  batchs 1210
TestSet : number of  batchs 151
ValSet : number of  batchs 152


In [7]:
trainset = trainset.cache().shuffle(1000).prefetch(buffer_size=tf.data.experimental.AUTOTUNE)
testset = testset.cache().shuffle(1000).prefetch(buffer_size=tf.data.experimental.AUTOTUNE)
valset = valset.cache().shuffle(1000).prefetch(buffer_size=tf.data.experimental.AUTOTUNE)

In [8]:
resize_and_rescale = tf.keras.Sequential([
    tf.keras.layers.experimental.preprocessing.Resizing(IMAGE_SIZE,IMAGE_SIZE),
    tf.keras.layers.experimental.preprocessing.Rescaling(1.0/255)
])

In [ ]:
model = CNN(3)

model.fit(trainset,epochs=6,batch_size=BATCH_SIZE,verbose=1,validation_data=valset)

Epoch 1/6


In [ ]:
import os
model_version = max([int(i) for i in os.listdir("models") + [0]]) + 1
model.save(f"models/PPT_{model_version}")

# Pepper

In [20]:
pepper = tf.keras.preprocessing.image_dataset_from_directory(
    "PlantVillage/Pepper",
    shuffle = True,
    image_size = (IMAGE_SIZE,IMAGE_SIZE),
    batch_size = BATCH_SIZE  
)

pepper.class_names

Found 3727 files belonging to 2 classes.


['Pepper__bell___Bacterial_spot', 'Pepper__bell___healthy']

In [21]:
print("Pepper Dataset Length : ",len(pepper))

trainset_pepper = pepper.take(63)
rest = pepper.skip(63)
testset_pepper = rest.take(8)
valset_pepper = rest.skip(8)

print("Pepper TrainSet length : ",len(trainset_pepper))
print("Pepper TestSet length : ",len(testset_pepper))
print("Pepper ValSet length : ",len(valset_pepper))

Pepper Dataset Length :  117
Pepper TrainSet length :  63
Pepper TestSet length :  8
Pepper ValSet length :  46


In [22]:
trainset_pepper = trainset_pepper.cache().shuffle(1000).prefetch(buffer_size=tf.data.experimental.AUTOTUNE)
testset_pepper = testset_pepper.cache().shuffle(1000).prefetch(buffer_size=tf.data.experimental.AUTOTUNE)
valset_pepper = valset_pepper.cache().shuffle(1000).prefetch(buffer_size=tf.data.experimental.AUTOTUNE)

In [24]:
model_pepper = CNN(2)
TrainCNN(model_pepper,trainset_pepper,valset_pepper,25)

Epoch 1/25
63/63 [==============================] - 205s 3s/step - loss: 0.5554 - accuracy: 0.7148 - val_loss: 0.3591 - val_accuracy: 0.8357
Epoch 2/25
63/63 [==============================] - 99s 2s/step - loss: 0.3639 - accuracy: 0.8805 - val_loss: 0.3239 - val_accuracy: 0.8426
Epoch 3/25
63/63 [==============================] - 100s 2s/step - loss: 0.3259 - accuracy: 0.8924 - val_loss: 0.2884 - val_accuracy: 0.9182
Epoch 4/25
63/63 [==============================] - 101s 2s/step - loss: 0.2389 - accuracy: 0.9147 - val_loss: 0.2113 - val_accuracy: 0.9251
Epoch 5/25
63/63 [==============================] - 102s 2s/step - loss: 0.2333 - accuracy: 0.9167 - val_loss: 0.2092 - val_accuracy: 0.9285
Epoch 6/25
63/63 [==============================] - 102s 2s/step - loss: 0.2285 - accuracy: 0.9221 - val_loss: 0.2276 - val_accuracy: 0.9100
Epoch 7/25
63/63 [==============================] - 102s 2s/step - loss: 0.2223 - accuracy: 0.9191 - val_loss: 0.2163 - val_accuracy: 0.9216
Epoch 8/25
63/

In [25]:
def save(model,name):
    model_version = max([i for i in range(len(os.listdir("models") + [0])) ]) + 1
    model.save(f"models/{name}{model_version}")

In [60]:
save(model_pepper,"Pepper")

INFO:tensorflow:Assets written to: models/Pepper2\assets


# Tomato

In [5]:
tomato = tf.keras.preprocessing.image_dataset_from_directory(
    "PlantVillage/Tomato",
    shuffle = True,
    image_size = (IMAGE_SIZE,IMAGE_SIZE),
    batch_size = BATCH_SIZE  
)

tomato.class_names

Found 16011 files belonging to 10 classes.


['Tomato_Bacterial_spot',
 'Tomato_Early_blight',
 'Tomato_Late_blight',
 'Tomato_Leaf_Mold',
 'Tomato_Septoria_leaf_spot',
 'Tomato_Spider_mites_Two_spotted_spider_mite',
 'Tomato__Target_Spot',
 'Tomato__Tomato_YellowLeaf__Curl_Virus',
 'Tomato__Tomato_mosaic_virus',
 'Tomato_healthy']

In [6]:
print("Tomato Dataset Length : ",len(tomato))

trainset_tomato = tomato.take(400)
rest = tomato.skip(400)
testset_tomato = rest.take(51)
valset_tomato = rest.skip(51)

print("Tomato TrainSet length : ",len(trainset_tomato))
print("Tomato TestSet length : ",len(testset_tomato))
print("Tomato ValSet length : ",len(valset_tomato))

Tomato Dataset Length :  501
Tomato TrainSet length :  400
Tomato TestSet length :  51
Tomato ValSet length :  50


In [7]:
trainset_tomato = trainset_tomato.cache().shuffle(1000).prefetch(buffer_size=tf.data.experimental.AUTOTUNE)
testset_tomato = testset_tomato.cache().shuffle(1000).prefetch(buffer_size=tf.data.experimental.AUTOTUNE)
valset_tomato = valset_tomato.cache().shuffle(1000).prefetch(buffer_size=tf.data.experimental.AUTOTUNE)

In [18]:
model_tomato = CNN(10)
TrainCNN(model_tomato,trainset_tomato,valset_tomato,10)

Epoch 1/10
400/400 [==============================] - 1054s 3s/step - loss: 1.5510 - accuracy: 0.4402 - val_loss: 0.8553 - val_accuracy: 0.7137
Epoch 2/10
400/400 [==============================] - 1148s 3s/step - loss: 0.7397 - accuracy: 0.7475 - val_loss: 0.5933 - val_accuracy: 0.7897
Epoch 3/10
400/400 [==============================] - 1294s 3s/step - loss: 0.4667 - accuracy: 0.8377 - val_loss: 0.4586 - val_accuracy: 0.8423
Epoch 4/10
400/400 [==============================] - 1195s 3s/step - loss: 0.3787 - accuracy: 0.8680 - val_loss: 0.2941 - val_accuracy: 0.8942
Epoch 5/10
400/400 [==============================] - 1226s 3s/step - loss: 0.2780 - accuracy: 0.9061 - val_loss: 0.2567 - val_accuracy: 0.9075
Epoch 6/10
400/400 [==============================] - 1337s 3s/step - loss: 0.2466 - accuracy: 0.9152 - val_loss: 0.2500 - val_accuracy: 0.9088
Epoch 7/10
400/400 [==============================] - 1246s 3s/step - loss: 0.2113 - accuracy: 0.9279 - val_loss: 0.2516 - val_accuracy:

In [19]:
# save(model_tomato,"Tomato")
model.save(f"models/Tomato2")

INFO:tensorflow:Assets written to: models/Tomato2\assets


# Potato

In [26]:
potato = tf.keras.preprocessing.image_dataset_from_directory(
    "PlantVillage/Potato",
    shuffle = True,
    image_size = (IMAGE_SIZE,IMAGE_SIZE),
    batch_size = BATCH_SIZE  
)

potato.class_names

Found 2152 files belonging to 3 classes.


['Potato___Early_blight', 'Potato___Late_blight', 'Potato___healthy']

In [27]:
print("Potato Dataset Length : ",len(potato))

trainset_potato = potato.take(54)
rest = potato.skip(54)
testset_potato = rest.take(7)
valset_potato = rest.skip(7)

print("Potato TrainSet length : ",len(trainset_potato))
print("Potato TestSet length : ",len(testset_potato))
print("Potato ValSet length : ",len(valset_potato))

Potato Dataset Length :  68
Potato TrainSet length :  54
Potato TestSet length :  7
Potato ValSet length :  7


In [28]:
trainset_potato = trainset_potato.cache().shuffle(1000).prefetch(buffer_size=tf.data.experimental.AUTOTUNE)
testset_potato = testset_potato.cache().shuffle(1000).prefetch(buffer_size=tf.data.experimental.AUTOTUNE)
valset_potato = valset_potato.cache().shuffle(1000).prefetch(buffer_size=tf.data.experimental.AUTOTUNE)

In [30]:
model_potato = CNN(3)
TrainCNN(model_potato,trainset_potato,valset_potato,25)

Epoch 1/25
54/54 [==============================] - 75s 1s/step - loss: 0.8896 - accuracy: 0.5330 - val_loss: 0.7817 - val_accuracy: 0.6650
Epoch 2/25
54/54 [==============================] - 75s 1s/step - loss: 0.6458 - accuracy: 0.7049 - val_loss: 0.4707 - val_accuracy: 0.7850
Epoch 3/25
54/54 [==============================] - 75s 1s/step - loss: 0.3760 - accuracy: 0.8507 - val_loss: 0.3501 - val_accuracy: 0.8750
Epoch 4/25
54/54 [==============================] - 75s 1s/step - loss: 0.2664 - accuracy: 0.8872 - val_loss: 0.3657 - val_accuracy: 0.8250
Epoch 5/25
54/54 [==============================] - 75s 1s/step - loss: 0.2176 - accuracy: 0.9248 - val_loss: 0.1730 - val_accuracy: 0.9400
Epoch 6/25
54/54 [==============================] - 75s 1s/step - loss: 0.1973 - accuracy: 0.9201 - val_loss: 0.2535 - val_accuracy: 0.9050
Epoch 7/25
54/54 [==============================] - 76s 1s/step - loss: 0.1407 - accuracy: 0.9456 - val_loss: 0.1472 - val_accuracy: 0.9450
Epoch 8/25
54/54 [==

In [31]:
model.save(f"models/Potato8")

INFO:tensorflow:Assets written to: models/Potato8\assets
